# Import Packages & Modules

In [1]:
import os
import random
import yaml

import re
from twilio.rest import Client
from twilio.base.version import TwilioRestException

In [2]:
from typing import (
    Dict,
    # Literal,
    List,
    Optional,
    TypeVar,
    # Union
)

## Define special ``types``

In [3]:
PhoneNumber = TypeVar('PhoneNumber', bound=str)
file = TypeVar('file', bound=str)
exit_status = TypeVar('exit_status', bound=int)

# Define classes

In [4]:
class InvalidPhoneNumber(Exception):
    """Exception for invalid phone numbers."""
    pass

# Define Functions

In [5]:
def read_file_to_str(infile: file) -> str:
    """Reads the contents of a file and returns a string.
    """
    infile: file = os.path.abspath(infile)
    with open(infile,'r') as f:
        t = f.readlines()
        return ' '.join(t).strip('\n')

In [6]:
def _format_phone_number(num: str) -> PhoneNumber:
    """Helper function that formats a string to resemble a phone number 
    for use with Twilio's ``python`` API.
    """
    num: PhoneNumber = re.sub(r'[^\w]','',num) # Remove special characters and white space
    
    if not num.startswith("1"):
        num: PhoneNumber = f"1{num}" # Check if phone number starts with US country code
    
    if not num.startswith("+"):
        num: PhoneNumber = f"+{num}" # Check if '+' is prepended to phone number
    
    if len(num) != 12:
        raise RuntimeError(f"Preprocessed phone number is not 12 characters long: {num}")
    
    return num

In [7]:
def send_sms(to: PhoneNumber,
             account_sid: str,
             service_sid: str,
             auth_token: str,
             body: Optional[str] = None,
             media: Optional[str] = None,
             outgoing: Optional[PhoneNumber] = None
            ) -> exit_status:
    """Utilizes the ``Twilio`` API to send SMS and/or MMS text messages to a known cell phone number.
    Messages sent successfully will have an exit status of ``0`` and ``1`` otherwise.

    NOTE: Account and service SIDs can be obtained from Twilio, alongside the authorization token.

    Usage example:
        >>> send_sms(to="+12346789012",
        ...          body="Hello world!")

    Arguments:
        to: The intended recipient of the text message. **NOTE**: The input phone number **MUST** be 11 digits and be prefixed with '+'.
        account_sid: ``Twilio`` API account service ID.
        service_sid: ``Twilio`` API messaging service ID.
        auth_token: ``Twilio`` API authentication token.
        body: The body of the text message (i.e. the message).
        media: HTTP link to multimedia (image, audio, video, ect).
        outgoing: ``Twilio`` account ``PhoneNumber`` to be used in place of ``service_sid``. **NOTE**: The input phone number **MUST** be 11 digits and be prefixed with '+'.

    Returns:
        exit_status: 0 if successful, 1 otherwise.

    Raises:
        RuntimeError: Arises **IF** the: ``auth_token`` is not specified, OR ``media`` and ``body`` are not specified 
            OR ``service_sid`` or ``outgoing`` are not specified.
    """
    # Check input arguments
    if (service_sid is None) and (outgoing is None):
        raise RuntimeError(f"Neither the 'service_sid' nor the 'outgoing' phone number were provided.")

    if auth_token is None:
        raise RuntimeError("No auth token provided.")

    if (media is None) and (body is None):
        raise RuntimeError("No text body OR media was specified as an argument.")

    # Init client
    client: Client = Client(account_sid, auth_token)

    # Construct SMS/MMS
    try:
        message: Client.messages = client.messages \
                    .create(
                         body=body,
                         media_url=media,
                         messaging_service_sid=service_sid,
                         from_=outgoing,
                         to=to
                     )
        exit_status: exit_status = 0
    except TwilioRestException:
        exit_status: exit_status = 1

    return exit_status

In [8]:
def read_config(config: file) -> Dict[str,str]:
    """Reads the contents of a YAML (configuration) file to
    a dictionary.

    Arguemnts:
        config: Filename (and path) to configuration file.
    
    Returns:
        Dictionary of strings matched valued pairs.
    """
    config: str = os.path.abspath(
        os.path.realpath(config)
        )
    
    with open(config,"r") as file:
        return yaml.safe_load(file)

In [15]:
def peer_match(config: file) -> Dict[str,str]:
    """Performs random peer-to-peer (person-to-person) matching.
    Peer exclusion is allowed if required with the ``exclude`` key
    if required.

    Arguments:
        config: Filename (and path) to configuration file.

    Returns:
        Dictionary with string matched pairs of peers.
    """
    # Read people config file
    ppl_dict: Dict[str,str] = read_config(config=config)

    # Set run condition
    run_condition: bool = True
    while run_condition:
        new_dict: Dict = {}
        assigned: List = []
        for p in ppl_dict.keys():
            recips = []
            if 'exclude' in ppl_dict.get(p):
                exclude: str = ppl_dict.get(p).get('exclude',None)

                # NOTE: Split comma separated names, flatten into one list
                exclude: List[str] = exclude.split(" , ")
                exclude: List[List[str]] = [name.split(", ") for name in exclude]; exclude: List[str] = [item for sublist in exclude for item in sublist]
                exclude: List[List[str]] = [name.split(" ,") for name in exclude]; exclude: List[str] = [item for sublist in exclude for item in sublist]
                exclude: List[List[str]] = [name.split(",") for name in exclude]; exclude: List[str] = [item for sublist in exclude for item in sublist]
                # exclude = list(exclude.split(","))

                recips: List[str] = list(set(ppl_dict.keys()) - set(exclude) - set(assigned))
            else:
                recips: List[str] = list(set(ppl_dict.keys()) - set(assigned))

            try:
                recips.remove(p)
            except ValueError:
                pass

            try:
                rand: int = random.randint(0, (len(recips))-1)
                random_recip: str = recips[rand]
                assigned.append(random_recip)
                tmp_dict: Dict[str,str] = {p: random_recip}
                new_dict.update(tmp_dict)
                del tmp_dict
                run_condition: bool = False
            except ValueError:
                run_condition: bool = True
                pass
    return new_dict

In [18]:
def gift_exchange(config: str,
                  account_sid: str,
                  service_sid: str,
                  auth_token: str,
                  dry_run: bool = False,
                  budget: float = 15.00,
                  year: int = 2021,
                  verbose: bool = False
                 ) -> Dict[str,exit_status]:
    """Randomized gift exchange function. Matches a random person
    to another person at random. The matched person is then texted,
    with information about the gift to be given and who the recipient is.
    
    Arguments:
        config: Configuration file that contains names, and phone numbers (and exclusion list(s)).
        account_sid: ``Twilio`` API account service ID.
        service_sid: ``Twilio`` API messaging service ID.
        auth_token: ``Twilio`` API authentication token.
        dry_run: If true, names and recipients are printed. If false, then text messages are sent.
        budget: Dollar amount for the budget.
        year: Year of the gift exchange.
        verbose: Enables verbose output to interpreter.
    
    Returns:
        Dictionary of pair matched strings to twilio exit statuses.
    """
    # Read in credentials
    if os.path.isfile(account_sid):
        account_sid: str = read_file_to_str(infile='account.sid')
    
    if os.path.isfile(service_sid):
        service_sid: str = read_file_to_str(infile='service.sid')
    
    if os.path.isfile(auth_token):
        auth_token: str = read_file_to_str(infile='auth.token')
    
    # Format budget
    budget: str = "${:,.2f}".format(float(budget))
    
    # Read information into dictionary
    people: Dict[str,Dict[str,str]] = read_config(config=config)
    person_match: Dict[str,str] = peer_match(config=config)

    runtime_status: Dict = {}

    # Iterate through peer-matched dictionaries and send
    #   text messages
    for person in people.keys():
        phone_num: PhoneNumber = _format_phone_number(num=str(people.get(person).get('phone')))
        peer: str = person_match.get(person)

        message: str = f"""Hi {person},
Welcome to the {int(year)} family gift exchange.
This year, you will be gifting to {peer}.
The recommended spending limit this year is {budget}.

Gifts will be exchanged at family dinner Thursday, Dec. 2, 2021.
        """

        if verbose:
            print(f"{person} gifts to {peer}.")
        
        if dry_run:
            status: exit_status = 1
            tmp_dict: Dict[str,str] = {person: status}
            runtime_status.update(tmp_dict)
        else:
            status: exit_status = send_sms(to=phone_num,
                                           account_sid=account_sid,
                                           service_sid=service_sid,
                                           auth_token=auth_token,
                                           body=message)
            
            tmp_dict: Dict[str,str] = {person: status}
            runtime_status.update(tmp_dict)
    return runtime_status

# Gift Exchange
-----

## Dry run of secret santa

In [28]:
# This file contains names and phone numbers
# conf: str = 'config.people.yml'

# This file contains FAKE names and phone numbers
conf: str = 'config.test.yml'

In [29]:
gift_exchange(config=conf,
            account_sid='account.sid',
            service_sid='service.sid',
            auth_token='auth.token',
            budget=10.00,
            year=2021,
            verbose=True,
            dry_run=True)

Person1 gifts to Person2.
Person2 gifts to Person1.


{'Person1': 1, 'Person2': 1}

## Send text messages for Secret Santa

In [41]:
# gift_exchange(config=conf,
#             account_sid='account.sid',
#             service_sid='service.sid',
#             auth_token='auth.token',
#             budget=10.00,
#             year=2021,
#             verbose=True,
#             dry_run=True)

Success: Email sent!
Success: Email sent!
Success: Email sent!
Success: Email sent!
